In [1]:
%load_ext autoreload
%autoreload 2

from mesa import Agent, Model
from mesa.time import StagedActivation
import numpy as np
from mesa.datacollection import DataCollector
import random
#from mesa.batchrunner import BatchRunner

In [5]:
%%writefile Agent.py
from mesa import Agent
from mesa.time import StagedActivation
from mesa.datacollection import DataCollector
from model import *
import numpy as np
import random

class CorruptionAgent(Agent):
    """ Agents in the model."""
    def __init__(self, unique_id, model, parent=None):
        """Create an instance of agents with properties:
        steps = 0, counter so that they can die
        parent = parent, if it was not spawned in the beginning
        b = risk-aversion, taken from a uniform distribution, according to the paper
        k = capital endowment, initially taken from a uniform distribution,
        then inherited from parent's income and taken from distribution."""
        super().__init__(unique_id, model)
        self.steps = 0 
        self.id = unique_id
        self.parent = parent
        self.get_risk_aversion()
        self.get_capital_endowment()
        #self.p=0
      
    def get_risk_aversion(self):
        self.b = np.random.uniform(self.model.b_bar-self.model.b_range/2,
                                   self.model.b_bar+self.model.b_range/2)
        
    def get_capital_endowment(self):
        if self.parent != None:
            self.k = (self.model.k_min + ((self.parent.y -self.model.y_min)/
                      (self.model.y_max - self.model.y_min)) *
                    (self.model.k_max-self.model.k_min) * (
                        1-self.model.theta) + 
                      (self.model.theta*np.random.uniform(
                          self.model.k_min, self.model.k_max)))
        else: 
            self.k = np.random.uniform(
                self.model.k_bar - self.model.k_range/2, 
                self.model.k_bar + self.model.k_range/2)

          
    def spawn_new_agent(self):
        """Spawn new agent with the spawning agent as parent, add to model."""
        new_agent=CorruptionAgent(self.unique_id, model, parent = self)
        self.model.schedule.add(new_agent)
        
             
    
    def choose_dishonesty_level(self):  
        """Choose a dishonesty level to maximize utility."""
        self.p = 1/(2*self.b*self.model.gamma**2*(
            1-self.model.q)*self.model.S)
        
    def get_income(self):
        """Get income y:
        One part is fixed, due to non-illicit corrupt work, 
        the other part due to corrupt work."""
        self.y = (1-self.p)*(1-self.model.q)*self.model.S*self.k + (
        self.p * np.random.normal(self.model.S*self.k, 
                                  (1-self.model.q)*self.model.gamma * 
                              self.model.S * self.k))
    def corrupt(self):
        ''' Choose dishonesty level, and associated income, add to steps.'''
        #check if
        self.choose_dishonesty_level()
        self.get_income()
        self.steps += 1
        
    def procreate(self):
        """ Breed new agent, then die."""
        self.spawn_new_agent()
        if self.steps >= 1:
            self.model.schedule.remove(self)


Overwriting Agent.py


In [3]:
%%writefile model.py 
from mesa import Agent, Model
from mesa.time import StagedActivation
import numpy as np
from mesa.datacollection import DataCollector
import random
from Agent import CorruptionAgent

def total_capital(model):
    """ Determine total capital of the model as sum of all the agents' capitals"""
    agent_capitals = [a.k for a in model.schedule.agents]
    capital = np.sum(agent_capitals)
    return capital

def corruption_index(model):
    """Determine total corruption index of the model. 
    Weighted average of all the dishonesty levels,
    with weights being each agent's capital"""
    capital = total_capital(model)
    corruption_index = 1/capital * (
        np.sum([a.k*a.p for a in model.schedule.agents]))
    return corruption_index
    
def min_max_income(model):
    """ Determine minimum and maximum income level each generation."""
    min_income = np.min([a.y for a in model.schedule.agents])
    max_income = np.max([a.y for a in model.schedule.agents])
    return min_income, max_income

def social_capital(model):
    """Determine social capital each generation."""
    capital = total_capital(model)
    social_capital = model.alpha * capital
    return social_capital

def national_income(model):
    """Determine the national income as sum over all the individual incomes."""
    incomes = [a.y for a in model.schedule.agents]
    national_income = np.sum(incomes)
    return national_income

class CorruptionModel(Model):
    """A model with population agents."""
    def __init__(self, b_bar = 3, b_range = 1, 
                 alpha= 0.5,  gamma = 0.5, theta= 0.1, q_start= 0.1, 
                 population=1000, k_bar = 0.5, 
                 k_range = 1):
        """Create the model with the following parameters:
        Average level of risk aversion = b_bar
        Range of risk aversion = b_range
        Proportion of income spent on vigilance = gamma
        Mean human capital endowment in first generation = k_bar
        Equality in access to human capital = theta
        Initial value of social corruption = q_start
        Population = population
        Number of generations = generations
        Range of human capital endowment: k_range"""
        #Set parameters
        self.running = True
        self.num_agents= population
        self.b_bar = b_bar
        self.b_range = b_range
        self.alpha = alpha
        self.gamma = gamma
        self.theta = theta
        self.q = q_start
        self.k_bar = k_bar
        self.k_range = k_range
        self.k_min = k_bar -0.5*k_range
        self.k_max = k_bar +0.5*k_range
        self.S = alpha * (k_bar * population)
        self.schedule = StagedActivation(self, stage_list=["corrupt", 
                                                           "procreate"])
        self.running = True
    
    
        #Create agents
        for i in range(self.num_agents):
            a = CorruptionAgent(i, self)
            self.schedule.add(a)       
              
        #Add data to report
        self.datacollector = DataCollector(
            model_reporters={"Total capital": total_capital, 
                             "Corruption Index": corruption_index, 
                            "National Income": national_income},
            agent_reporters={"Dishonesty": lambda a:a.p})
        
        
        
    def step(self):
        ''' Advance the model by one step. Do this in two stages:
        In first stage, corrupt, then report data and update model parameters.
        In second stage, breed and die.'''
        for stage in self.schedule.stage_list:
            for agent in self.schedule.agents[:]:
                getattr(agent, stage)() 
            if stage == "corrupt":
                self.q = corruption_index(self)
                self.datacollector.collect(self)
                self.y_min, self.y_max = min_max_income(self)
                #self.k_min, self.k_max = min_max_capital(self)
            self.K = total_capital(self)
            self.S = social_capital(self)
    
                
    

Overwriting model.py


In [6]:
from model import *
from Agent import CorruptionAgent

model = CorruptionModel()

for i in range(100):
    model.step()


NameError: name 'model' is not defined

In [ ]:
model.datacollector.get_model_vars_dataframe()

In [ ]:
model.datacollector.get_agent_vars_dataframe()

params = {"population": 1000, 
                 "alpha" : 0.5,  
                "gamma" : 0.5, 
                "theta":  0.1, 
                "q_start" : 0.1, 
                 "k_bar" : 0.5, 
                 "k_range" : 1,
               "b_bar": [2, 3, 4],
                   "b_range" : 1}
    #"b_bar" : [3, 3.5, 4]}
batch_run = BatchRunner(CorruptionModel, 
                        parameter_values = params,
                        
                        iterations = 10, 
                        max_steps = 5, 
                        model_reporters = {
                            "Corruption Index": corruption_index, 
                            "National Income": national_income}, 
                        display_progress =True)